In [ ]:
!pip install yfinance
!mamba install bs4
!pip install nbformat
!pip install pandas_datareader
!pip install mplfinance

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import mplfinance as mpf 

In [ ]:
reliance = yf.Ticker("MRF.NS")

# Get stock info
print(reliance.info)

# Fetch historical data (last 5 days)
historical_data = reliance.history(period="5d")
print(historical_data)

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL for Reliance Industries financial statements on Yahoo Finance
url = "https://finance.yahoo.com/quote/MRF.NS/financials"

# Set headers to avoid request blocking
headers = {"User-Agent": "Mozilla/5.0"}

# Send a GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse HTML data
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract financial table data
    financial_data = soup.find_all("div", {"data-test": "fin-row"})

    # Print extracted data
    for row in financial_data:
        print(row.get_text())

else:
    print("Failed to retrieve data. Status Code:", response.status_code)


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import Image



# Fetch stock data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    stock_data = stock.history(period="6mo")
    return stock_data
from fpdf import FPDF

def create_report(ticker, stock_data, pdf):
    # Ensure 'Close' column is selected and convert to numeric
    price_column = 'Adj Close' if 'Adj Close' in stock_data.columns else 'Close'
    stock_data[price_column] = pd.to_numeric(stock_data[price_column], errors='coerce')

    # Extract high & low prices
    highest_price = stock_data['High'].max()
    lowest_price = stock_data['Low'].min()
    start_date = stock_data.index.min().date()
    end_date = stock_data.index.max().date()

    # Report text
    report_text = f"""
    Stock Report for {ticker}
    ------------------------------
    - Time Period: {start_date} to {end_date}
    - Highest Price: {highest_price:.2f}
    - Lowest Price: {lowest_price:.2f}
    - Mean Closing Price: {stock_data[price_column].mean():.2f}
    - Standard Deviation: {stock_data[price_column].std():.2f}
    """

    print(report_text)  # Check if the report text is generated correctly

    # Add Title
    pdf.set_font("Arial", style='B', size=16)
    pdf.cell(200, 10, f"Stock Report: {ticker}", ln=True, align='C')

    # Add Report Text to PDF
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, report_text)

    # Save PDF
    pdf.output(f"{ticker}_Stock_Report.pdf")



# Generate plots
def generate_plots(stock_data, ticker):
    # Save closing price chart
    plt.figure(figsize=(10, 5))
    plt.plot(stock_data.index, stock_data['Close'], label='Close Price', color='blue')
    plt.title(f'{ticker} Closing Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig('closing_price.png')
    plt.close()
    
    # Save candlestick chart
    mpf.plot(stock_data, type='candle', style='charles', volume=True, savefig='candlestick_chart.png')
    
    # Moving average chart
    stock_data['Short_MA'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['Long_MA'] = stock_data['Close'].rolling(window=50).mean()
    plt.figure(figsize=(10, 5))
    plt.plot(stock_data.index, stock_data['Close'], label='Close Price', color='blue')
    plt.plot(stock_data.index, stock_data['Short_MA'], label='20-Day MA', linestyle='dashed', color='red')
    plt.plot(stock_data.index, stock_data['Long_MA'], label='50-Day MA', linestyle='dashed', color='green')
    plt.title(f'{ticker} Moving Averages')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig('moving_average.png')
    plt.close()
    
    # Volume vs. Price chart
    fig, ax1 = plt.subplots(figsize=(10, 5))
    ax1.plot(stock_data.index, stock_data['Close'], color='blue', label='Close Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price', color='blue')
    
    ax2 = ax1.twinx()
    ax2.bar(stock_data.index, stock_data['Volume'], color='gray', alpha=0.3, label='Volume')
    ax2.set_ylabel('Volume')
    
    fig.suptitle(f'{ticker} Price vs. Volume')
    fig.tight_layout()
    plt.savefig('volume_vs_price.png')
    plt.close()

# Generate PDF report
def generate_pdf_report(ticker):
    pdf = canvas.Canvas(f"{ticker}_financial_report.pdf", pagesize=letter)
    pdf.drawString(200, 750, f"{ticker} Financial Report")
    
    # Insert plots into PDF
    pdf.drawImage("closing_price.png", 50, 500, width=500, height=200)
    pdf.drawImage("candlestick_chart.png", 50, 250, width=500, height=200)
    pdf.showPage()
    pdf.drawImage("moving_average.png", 50, 500, width=500, height=200)
    pdf.drawImage("volume_vs_price.png", 50, 250, width=500, height=200)
    pdf.showPage()
    pdf.save()
    print(f"PDF report {ticker}_financial_report.pdf generated successfully.")

# Main function
def main():
    ticker = "MRF.NS"
    stock_data = fetch_stock_data(ticker)
    generate_plots(stock_data, ticker)
    generate_pdf_report(ticker)

if __name__ == "__main__":
    main()
